In [1]:
import torch
import json
from tqdm import tqdm
from itertools import product as p
import numpy as np

# Load fake activations

In [2]:
class A:
    pass
self = A()

In [3]:
self.num_neurons_d = {} # {fname, int}
self.representations_d = {} # {fname, tensor}
f1, f2, f3 = "foo", "bar", "baz"
representation_files = [f1, f2, f3]

In [4]:
# initialize `num_neurons_d`, `representations_d` with fake data
n1, n2, n3 = 100, 80, 70
nword = 1000
t1 = torch.randn(nword, n1)
t2 = torch.randn(nword, n2)
t3 = torch.randn(nword, n3)
self.num_neurons_d[f1] = n1
self.num_neurons_d[f2] = n2
self.num_neurons_d[f3] = n3
self.representations_d[f1] = t1
self.representations_d[f2] = t2
self.representations_d[f3] = t3

In [5]:
self.normalize_dimensions = True

# Build function

### Normalize

In [6]:
# Normalize
if self.normalize_dimensions:
    for network in tqdm(self.representations_d, desc='mu, sigma'):
        # TODO: might not need to normalize, only center
        t = self.representations_d[network]
        means = t.mean(0, keepdim=True)
        stdevs = t.std(0, keepdim=True)

        self.representations_d[network] = (t - means) / stdevs

mu, sigma: 100%|█| 3/3 [00:00<00:00, 389.94it/s]


### Set `self.similarities`

In [8]:
self.similarities = {network: {} for network in self.representations_d}

In [7]:
# loop variables
network = f1
other_network = f2

In [ ]:
# if network == other_network:
#     continue

# if other_network in self.transforms[network].keys(): # TO DO: optimize?
#     continue

In [9]:
X = self.representations_d[network]
Y = self.representations_d[other_network]

In [16]:
XtX_F = torch.norm(torch.mm(X.t(), X), p='fro').item()
YtY_F = torch.norm(torch.mm(Y.t(), Y), p='fro').item()
YtX_F = torch.norm(torch.mm(Y.t(), X), p='fro').item()

In [17]:
self.similarities[network][other_network] = YtX_F**2 / (XtX_F*YtY_F)

In [19]:
# full
self.similarities = {network: {} for network in self.representations_d}
for network, other_network in tqdm(p(self.representations_d,
                                     self.representations_d), desc='cka',
                                   total=len(self.representations_d)**2):

    if network == other_network:
        continue

    if other_network in self.similarities[network].keys(): # TO DO: optimize?
        continue

    X = self.representations_d[network]
    Y = self.representations_d[other_network]

    XtX_F = torch.norm(torch.mm(X.t(), X), p='fro').item()
    YtY_F = torch.norm(torch.mm(Y.t(), Y), p='fro').item()
    YtX_F = torch.norm(torch.mm(Y.t(), X), p='fro').item()

    self.similarities[network][other_network] = YtX_F**2 / (XtX_F*YtY_F)

cca: 100%|█| 9/9 [00:00<00:00, 739.45it/s]


# Build full function

In [21]:
def compute_correlations(self):
    """
    Set `self.similarities`. 
    """
    # Normalize
    if self.normalize_dimensions:
        for network in tqdm(self.representations_d, desc='mu, sigma'):
            # TODO: might not need to normalize, only center
            t = self.representations_d[network]
            means = t.mean(0, keepdim=True)
            stdevs = t.std(0, keepdim=True)

            self.representations_d[network] = (t - means) / stdevs

    # Set `self.similarities`
    # {network: {other: cka_similarity}}
    self.similarities = {network: {} for network in self.representations_d}
    for network, other_network in tqdm(p(self.representations_d,
                                         self.representations_d), desc='cka',
                                       total=len(self.representations_d)**2):

        if network == other_network:
            continue

        if other_network in self.similarities[network].keys(): # TO DO: optimize?
            continue

        X = self.representations_d[network]
        Y = self.representations_d[other_network]

        XtX_F = torch.norm(torch.mm(X.t(), X), p='fro').item()
        YtY_F = torch.norm(torch.mm(Y.t(), Y), p='fro').item()
        YtX_F = torch.norm(torch.mm(Y.t(), X), p='fro').item()

        self.similarities[network][other_network] = YtX_F**2 / (XtX_F*YtY_F)

In [23]:
compute_correlations(self)
self.similarities

cca: 100%|█| 9/9 [00:00<00:00, 646.50it/s]


{'foo': {'bar': 0.0836337778605439, 'baz': 0.07792939252860141},
 'bar': {'foo': 0.08363383516681212, 'baz': 0.06839857736053304},
 'baz': {'foo': 0.07792963611245719, 'bar': 0.06839859161987433}}